In [ ]:
!echo $SHELL
!echo $PATH
!echo $JOURNAL_ROOT

%load_ext autoreload
%autoreload 2
import logging

logging.basicConfig(level=logging.ERROR)
import random

random.seed(0)
import os
import pandas as pd

import dotenv

dotenv.load_dotenv()

import pprint

pprint.pp(dict(os.environ))

os.getenv("JOURNAL_ROOT")


pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

!rm -rf output
logger = logging.getLogger()


logger.setLevel(logging.ERROR)
logging.getLogger().setLevel(logging.WARNING)


def mode_debug():
    logging.getLogger().setLevel(logging.DEBUG)


def mode_info():
    logging.getLogger().setLevel(logging.INFO)


def mode_warn():
    logging.getLogger().setLevel(logging.WARN)


def mode_err():
    logging.getLogger().setLevel(logging.ERROR)


In [ ]:
# src/journal.py
"""
This file is used to define some rudimentray functions for test data and also the actual journal with real data.

The main goal is to use the features.py file to define features step by step for each trade and then do little to no changes afterwards, besides maybe adding 
new features afterwards. The trades are then stored in a journal, which is simply a list of trades.
The journal supports going over all the trades and creating a feature dataframe for further analysis.

"""

import os
import random
import logging

# MY LIBRARIES
import trade
import analysis

logging.basicConfig(level=logging.INFO)

# TRADING ACCOUNTS
ACC_IDEAL = "ideal"
ACC_MT5_VANTAGE = "mt5_vantage"
ACC_TEST = "test_account"

poi_sc_1h_mitigation = "POI_SC_1h_mitigation"
poi_sc_15m_mitigation = "POI_SC_15m_mitigation"
poi_sc_5m_mitigation = "POI_SC_5m_mitigation"

todo = "TODO"

sugar = "sugar"
sugar_values = ["big_fu_candle_bullish", "big_fu_candle_bearish"]

_trades = []

# Trade 01
tr01 = {"uid": "01"}
t = trade.Trade(
    entry_price=85490.33,
    size=0.5,
    entry_time="2025-04-15 13:11:29",
    side="long",
    sl_price=85379.48,
    tp_price=90068.42,
    sl_monetary_value=-48.70,
)

t.add_position(
    entry_price=85557.41, size=0.5, entry_time="2025-04-15 13:25:18"
)  # corrected time format

# full close
t.close_position(exit_price=85578, exit_time="2025-04-15 14:39:00")
tr01.update(t.to_trade_row())
tr01.update(
    {
        "POI_1h_SC": True,
        "POI_1h_AOI": True,
        "POI_15m_liqgrb": False,
        "EM_1m_type3": True,
        "RM": "1R_BE",
        "BEHAVIOUR": [("CONSOLIDATION", False)],
    }
)
_trades.append(tr01)

# Trade 02
tr02 = {"uid": "02"}
t = trade.Trade(
    entry_price=85490.33,
    size=0.5,
    entry_time="2025-04-15 13:11:29",
    side="long",
    sl_price=85379.48,
    tp_price=90068.42,
    sl_monetary_value=-48.70,
)

t.add_position(entry_price=85557.41, size=0.5, entry_time="2025-04-15 13:25:18")

t.close_position(exit_price=85578, exit_time="2025-04-15 14:39:00")
tr02.update(t.to_trade_row())
tr02.update(
    {
        "POI_1h_SC": True,
        "POI_1h_AOI": False,
        "POI_15m_liqgrb": True,
        "EM_1m_type3": False,
        "EM_POST_1m_type3": True,
        "RM": "1R_BE",
        "BEHAVIOUR": [("CONSOLIDATION", True)],
    }
)
_trades.append(tr02)

# Trade 03 (copy style)
tr03 = {"uid": "03"}
t = trade.Trade(
    entry_price=85490.33,
    size=0.5,
    entry_time="2025-04-15 13:11:29",
    side="long",
    sl_price=85379.48,
    tp_price=90068.42,
    sl_monetary_value=-48.70,
)

t.add_position(entry_price=85557.41, size=0.5, entry_time="2025-04-15 13:25:18")

t.close_position(exit_price=85578, exit_time="2025-04-15 14:39:00")
tr03.update(t.to_trade_row())
tr03.update(
    {
        "POI_1h_SC": True,
        "POI_1h_AOI": False,
        "POI_15m_liqgrb": True,
        "EM_1m_type3": False,
        "EM_POST_1m_type3": True,
        "RM": "1R_BE",
        "BEHAVIOUR": [("CONSOLIDATION", True)],
    }
)
_trades.append(tr03)

# CPI news trade 12-August trade on NAS (uid 04)
tr04 = {"uid": "04"}
news_trade = trade.Trade(
    entry_price=23000,
    size=0.1,
    entry_time="2025-08-12 13:11:29",
    side="long",
    sl_price=22900,
    tp_price=23500,
    sl_monetary_value=-5,
)

# partial + remaining close demonstration
news_trade.close_position(size=0.05, exit_price=23100, exit_time="2025-08-12 13:45:00")
news_trade.close_position(exit_price=23200, exit_time="2025-08-12 14:39:00")
tr04.update(news_trade.to_trade_row())
tr04.update({"TODO": True})
_trades.append(tr04)

df = pd.DataFrame(_trades)
df["account"] = ACC_MT5_VANTAGE

display(df.tail(len(_trades)))
display(df.describe(include="all"))

print(df.columns)
print(df.dtypes)

# report Nans for each column
print("Nans:")
print(df.isna().sum())

# report Nans for each column as percentage
print("Nans as percentage:")
print(df.isna().sum() / df.shape[0])

In [ ]:
mode_warn()


def prepare_data(df: pd.DataFrame):
    df = df.rename(
        columns={"return": "return_points", "initial_entry_time": "entry_time"}
    )
    df = df[(df["account"] != ACC_TEST) & (True)]

    return df


df = prepare_data(df)


df

In [ ]:
analysis.plot_feature_distributions(
    df,
    ["POI_1h_SC", "POI_1h_AOI", "POI_15m_liqgrb", "EM_1m_type3"],
)

In [ ]:
df

In [ ]:
t = news_trade  # use the last constructed trade with partial + full close

state_df = pd.DataFrame(t.get_trade_summary())
display(state_df)
summary_row = t.to_trade_row()
display(pd.DataFrame([summary_row]))
display(t.get_state_history())

chart = t.plot_trade_levels_altair()
chart

In [ ]:
# Inspect relative execution sizes to confirm sizing logic
state_df[["time", "act", "exec_size", "realised_profit"]]


In [ ]:
state_df.columns